In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of USFWS biologist labels of imagery from Bosque del Apache and Maxwell NWR

In [30]:
#Imports
import pandas as pd

import sklearn.metrics

import numpy as np
from shapely.geometry import Polygon,Point
import matplotlib.pyplot as plt
import shapely
import cv2 as cv
import os
import gc

In [3]:
#Analysis annotations
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_expertanalysislabels.csv"
with open(path) as f:
  df = pd.read_csv(f)

#Refined annotations
refinedpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_expertconsensuslabels.csv"
with open(refinedpath) as f:
  ref = pd.read_csv(f)

In [9]:
df = df[df["filename"] != "BDA_24C_20181107_1.JPG"]
df["filename"].unique()

array(['BDA_12C_20181127_1.JPG', 'BDA_12C_20181127_2.JPG',
       'BDA_12C_20181127_3.JPG', 'BDA_18A4_20181106_1.JPG',
       'BDA_18A4_20181106_2.JPG', 'BDA_18A4_20181106_3.JPG',
       'BDA_18A4_20181106_4.JPG', 'BDA_18A4_20181107_1.JPG',
       'BDA_18A4_20181107_2.JPG', 'BDA_18A4_20181107_3.JPG',
       'BDA_18A4_20181107_4.JPG', 'mxw_L13_20181215_1.JPG'], dtype=object)

In [10]:
clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()
c = clusters.reset_index()
c["expected"] = c["bbox_orig"]/c["labeler"]
c["diff"] = c["cluster_id"]/c["expected"]
clusters.to_csv()
#pd.eval("c['diff'] > 1.1 or c['diff'] < 0.9").sum()

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_23201/3555870435.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()


,bbox_orig,cluster_id,labeler
filename,,,
BDA_12C_20181127_1.JPG,841,86,10
BDA_12C_20181127_2.JPG,3817,489,8
BDA_12C_20181127_3.JPG,4867,793,6
BDA_18A4_20181106_1.JPG,1622,164,10
BDA_18A4_20181106_2.JPG,3497,356,10
BDA_18A4_20181106_3.JPG,438,63,7
BDA_18A4_20181106_4.JPG,1145,116,10
BDA_18A4_20181107_1.JPG,780,79,10
BDA_18A4_20181107_2.JPG,201,27,7


In [37]:
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/"
#exportanalysis = datetime.now().strftime('%Y%m%d_zooniverseanalysislabels_seagull_nodrops.csv')
clusters.to_csv(path+"clustercheck.csv")

In [12]:
def minmax(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (min_val, max_val)

print(minmax(c["diff"]))
print(c["diff"].std())
print(c["diff"].var())

(0.9402985074626865, 1.0248886560125754)
0.02409051426821127
0.0005803528777068907


In [6]:
#Refined annotation statistics
print(len(ref))
print(ref["category"].value_counts())

2375
Mallard                                                               1688
Northern Pintail                                                       262
Canadian Goose                                                         140
Other                                                                   70
Sandhill Crane                                                          52
['Mallard' 'Other']                                                     33
American Wigeon                                                         22
['Mallard' 'Northern Pintail']                                          20
['Northern Pintail' 'Other']                                            15
['American Wigeon' 'Mallard']                                           15
['American Wigeon' 'Mallard' 'Northern Pintail' 'Other' 'Teal']          9
['American Wigeon' 'Other']                                              7
Gadwall                                                                  5
['Gadwall' 'Mallard'

In [15]:
#Cluster reliability: Group by filename, labeler-- count the number of original bounding boxes per labeler, get range/variance

df.groupby("labeler")["bbox_orig"].count()


labeler
andrew_stetter@fws.gov             1145
barry_wilson@fws.gov               2315
bill_johnson@fws.gov               1983
dan_collins@fws.gov                1395
david.butler@tpwd.texas.gov        2747
jeff_sanchez@fws.gov               1791
jena_moon@fws.gov                  1076
john_vradenburg@fws.gov            1426
josh_vest@fws.gov                   508
jude_smith@fws.gov                 1844
kammie_kruse@fws.gov                960
mbrasher@ducks.org                  485
ronald_deroche@fws.gov              485
stephen.mcdowell@tpwd.texas.gov    1096
steven_sesnie@fws.gov                68
Name: bbox_orig, dtype: int64

In [40]:
range = df.groupby(['filename', 'labeler']).size()
r = range.reset_index()

r.rename( columns={0 :'count'}, inplace=True )
r.groupby("filename")["count"].apply(minmax)


filename
BDA_12C_20181127_1.JPG        (68, 93)
BDA_12C_20181127_2.JPG      (288, 570)
BDA_12C_20181127_3.JPG     (220, 1043)
BDA_18A4_20181106_1.JPG     (159, 165)
BDA_18A4_20181106_2.JPG     (300, 367)
BDA_18A4_20181106_3.JPG       (62, 66)
BDA_18A4_20181106_4.JPG     (113, 117)
BDA_18A4_20181107_1.JPG       (75, 83)
BDA_18A4_20181107_2.JPG       (26, 36)
BDA_18A4_20181107_3.JPG       (88, 91)
BDA_18A4_20181107_4.JPG       (83, 86)
mxw_L13_20181215_1.JPG        (38, 39)
Name: count, dtype: object

In [55]:
#Make unique cluster IDs and map to the original file 
#spplist = ['Canadian Goose', 'Sandhill Crane', 'Mallard','Northern Pintail','American Wigeon','Other','Teal','Gadwall','Northern Shoveler']
ref["category"]


0       Canadian Goose
1       Canadian Goose
2       Canadian Goose
3       Canadian Goose
4       Canadian Goose
             ...      
2370    Canadian Goose
2371    Canadian Goose
2372    Canadian Goose
2373    Canadian Goose
2374    Canadian Goose
Name: category, Length: 2375, dtype: object

In [ ]:
#INDIVIDUAL LABELER METRICS
#Goal: report avg count, ID consensus across images, break down by morphological group; also min/max individual vs group consensus
#Positional error: create confusion matrix, report commission error, omission error, RMSE, MAE -- bounding boxes, comparing each labeler to the consensus

sklearn.metrics.confusion_matrix(true,pred)

#Score agreement with consensus class label

#Score %IOU agreement with consensus box 

In [31]:
#Confusion matrix for bounding boxes solution from: https://towardsdatascience.com/confusion-matrix-and-object-detection-f0cbcb634157

def evaluation(ground,pred,iou_value):
  """
  ground= array of ground-truth contours.
  preds = array of predicted contours.
  iou_value= iou treshold for TP and otherwise.
  """
  truth=np.squeeze(ground)
  preds=np.squeeze(pred)
  #we will use this function to check iou less than threshold
  def CheckLess(list1,val):
    return(all(x<=val for x in list1))

  # Using predicted output as the reference
  prob1=[]
  for i in range(len(preds)):
      f1=preds[i]
      # define a Shapely polygone for prediction i
      f1=shapely.geometry.Polygon(f1)
      # determine the radius
      f1_radius=np.sqrt((f1.area)/np.pi)
      #buffer the polygon fromt the centroid
      f1_buffered=shapely.geometry.Point(f1.centroid).buffer(f1_radius*500)
      cont=[]
      for i in range(len(truth)):
        ff=shapely.geometry.Polygon(np.squeeze(truth[i]))
        if f1_buffered.contains(ff)== True:
          iou=(ff.intersection(f1).area)/(ff.union(f1).area)  
       
          cont.append((iou))

      prob1.append(cont)

  fp=0

  for t in prob1:
    if CheckLess(t,iou_value)==True:
      fp=fp+1
    
  prob2=[]
  #loop through each groun truth instance 
  for i in range(len(truth)):
      f1=truth[i]
      f1=shapely.geometry.Polygon(f1)
      #find radius
      f1_radius=np.sqrt((f1.area)/np.pi)
      #buffer the polygon from the centroid
      f1_buffered=shapely.geometry.Point(f1.centroid).buffer(f1_radius*500)
      cont=[]
      # merge up the ground truth instance against prediction
      # to determine the IoU
      for i in range(len(preds)):
        ff=shapely.geometry.Polygon(np.squeeze(preds[i]))
        if f1_buffered.contains(ff)== True:
          #calculate IoU
          iou=(ff.intersection(f1).area)/(ff.union(f1).area)
          cont.append((iou))
      # probability of a given prediction to be contained in a
      # ground truth instance
      prob2.append(cont)
  fn=0
  tp=0
  for t in prob2:
    if np.sum(t)==0:
      fn=fn+1
    elif CheckLess(t,iou_value)==False:
      tp=tp+1
  
  #lets add this section just to print the results
  print("TP:",tp,"\t FP:",fp,"\t FN:",fn,"\t GT:",truth.shape[0])
  precision=round(tp/(tp+fp),3) 
  recall=round(tp/(tp+fn),3)
  f1= round(2*((precision*recall)/(precision+recall)),3)
  print("Precall:",precision,"\t Recall:",recall, "\t F1 score:",f1)
  
  return tp,fp,fn,precision,recall,f1

In [35]:
pred = df["bbox_orig"]
truth = df["bbox_refined"]
iou = 0.3
df.head()

,Unnamed: 0,annotation_ID,bbox_orig,filename,labeler,cat_orig,cluster_id,cat_refined,bbox_refined
0,0,1,"[4428, 2707, 125, 103]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,0,Canadian Goose,"[4445.5, 2719.5, 95.0, 80.5]"
1,1,2,"[4308, 2731, 105, 67]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,1,Canadian Goose,"[4312.5, 2739.5, 98.0, 44.0]"
2,2,3,"[3707, 1761, 110, 101]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,2,Canadian Goose,"[3725.5, 1779.0, 73.5, 70.5]"
3,3,4,"[3628, 1882, 90, 38]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,3,Canadian Goose,"[3628.0, 1882.0, 92.0, 38.0]"
4,4,5,"[3669, 1927, 69, 82]",BDA_12C_20181127_1.JPG,steven_sesnie@fws.gov,Canadian Goose,4,Canadian Goose,"[3679.0, 1929.0, 65.0, 82.0]"


In [34]:
evaluation(truth,pred,iou)

TypeError: 'Series' object is not callable

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from object_detection.core import standard_fields
from object_detection.metrics import tf_example_parser
from object_detection.utils import label_map_util

In [36]:


flags = tf.app.flags

flags.DEFINE_string('label_map', None, 'Path to the label map')
flags.DEFINE_string('detections_record', None, 'Path to the detections record file')
flags.DEFINE_string('output_path', None, 'Path to the output the results in a csv.')

FLAGS = flags.FLAGS

IOU_THRESHOLD = 0.5
CONFIDENCE_THRESHOLD = 0.5

def compute_iou(groundtruth_box, detection_box):
    g_ymin, g_xmin, g_ymax, g_xmax = tuple(groundtruth_box.tolist())
    d_ymin, d_xmin, d_ymax, d_xmax = tuple(detection_box.tolist())
    
    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)

    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)

    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)

    return intersection / float(boxAArea + boxBArea - intersection)

def process_detections(detections_record, categories):
    record_iterator = tf.python_io.tf_record_iterator(path=detections_record)
    data_parser = tf_example_parser.TfExampleDetectionAndGTParser()

    confusion_matrix = np.zeros(shape=(len(categories) + 1, len(categories) + 1))

    image_index = 0
    for string_record in record_iterator:
        example = tf.train.Example()
        example.ParseFromString(string_record)
        decoded_dict = data_parser.parse(example)
        
        image_index += 1
        
        if decoded_dict:
            groundtruth_boxes = decoded_dict[standard_fields.InputDataFields.groundtruth_boxes]
            groundtruth_classes = decoded_dict[standard_fields.InputDataFields.groundtruth_classes]
            
            detection_scores = decoded_dict[standard_fields.DetectionResultFields.detection_scores]
            detection_classes = decoded_dict[standard_fields.DetectionResultFields.detection_classes][detection_scores >= CONFIDENCE_THRESHOLD]
            detection_boxes = decoded_dict[standard_fields.DetectionResultFields.detection_boxes][detection_scores >= CONFIDENCE_THRESHOLD]
            
            matches = []
            
            if image_index % 100 == 0:
                print("Processed %d images" %(image_index))
            
            for i in range(len(groundtruth_boxes)):
                for j in range(len(detection_boxes)):
                    iou = compute_iou(groundtruth_boxes[i], detection_boxes[j])
                    
                    if iou > IOU_THRESHOLD:
                        matches.append([i, j, iou])
                    
            matches = np.array(matches)
            if matches.shape[0] > 0:
                # Sort list of matches by descending IOU so we can remove duplicate detections
                # while keeping the highest IOU entry.
                matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]
                
                # Remove duplicate detections from the list.
                matches = matches[np.unique(matches[:,1], return_index=True)[1]]
                
                # Sort the list again by descending IOU. Removing duplicates doesn't preserve
                # our previous sort.
                matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]
                
                # Remove duplicate ground truths from the list.
                matches = matches[np.unique(matches[:,0], return_index=True)[1]]
                
            for i in range(len(groundtruth_boxes)):
                if matches.shape[0] > 0 and matches[matches[:,0] == i].shape[0] == 1:
                    confusion_matrix[groundtruth_classes[i] - 1][detection_classes[int(matches[matches[:,0] == i, 1][0])] - 1] += 1 
                else:
                    confusion_matrix[groundtruth_classes[i] - 1][confusion_matrix.shape[1] - 1] += 1
                    
            for i in range(len(detection_boxes)):
                if matches.shape[0] > 0 and matches[matches[:,1] == i].shape[0] == 0:
                    confusion_matrix[confusion_matrix.shape[0] - 1][detection_classes[i] - 1] += 1
        else:
            print("Skipped image %d" % (image_index))

    print("Processed %d images" % (image_index))

    return confusion_matrix

def display(confusion_matrix, categories, output_path):
    print("\nConfusion Matrix:")
    print(confusion_matrix, "\n")
    results = []

    for i in range(len(categories)):
        id = categories[i]["id"] - 1
        name = categories[i]["name"]
        
        total_target = np.sum(confusion_matrix[id,:])
        total_predicted = np.sum(confusion_matrix[:,id])
        
        precision = float(confusion_matrix[id, id] / total_predicted)
        recall = float(confusion_matrix[id, id] / total_target)
        
        #print('precision_{}@{}IOU: {:.2f}'.format(name, IOU_THRESHOLD, precision))
        #print('recall_{}@{}IOU: {:.2f}'.format(name, IOU_THRESHOLD, recall))
        
        results.append({'category' : name, 'precision_@{}IOU'.format(IOU_THRESHOLD) : precision, 'recall_@{}IOU'.format(IOU_THRESHOLD) : recall})
    
    df = pd.DataFrame(results)
    print(df)
    df.to_csv(output_path)
    
def main(argv):
    del argv
    required_flags = ['detections_record', 'label_map', 'output_path']
    for flag_name in required_flags:
        if not getattr(FLAGS, flag_name):
            raise ValueError('Flag --{} is required'.format(flag_name))

    label_map = label_map_util.load_labelmap(FLAGS.label_map)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=100, use_display_name=True)

    confusion_matrix = process_detections(FLAGS.detections_record, categories)

    display(confusion_matrix, categories, FLAGS.output_path)    
    
if __name__ == '__main__':
    tf.app.run(main)

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
#OUTPUTS

#Line graph of average identification consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Bar graph of consensus (y; % agreement) by morphology (x; duck/goose/crane). Two bars per class, one for expert, one for crowdsourced
#Line graph of average count consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Line graph of consensus (y; % agreement) by density (x; consensus # of individuals per image). One line for expert, one line for crowdsourced.
